In [1]:
%run nb-env.ipynb
pd.set_option("display.precision", 2)

# TODO: check dates

[2022-03-20T12:40:30.273681+08:00] Enviroment loaded. Working Dir: /Users/tzhu/work/lab/neo_world/notebooks
Today is 2022-03-20, Working from 2021-03-03 to 2022-03-18


In [2]:
today_rs = pdl.now().to_date_string()
today_rs

'2022-03-20'

In [10]:
%%time

dc, df_init = init_data(biquater_ago_date, end_date, expire_days=5)
df = df_init.copy()
clean_cache_files()

Initializing data from 2021-08-12 to 2022-03-14...
Found cache file: ../tmp/price_2021-08-12_2022-03-14_2022-03-19.feather, loading...
Preserving recent file price_2020-01-29_2021-10-29_2022-12-31.feather
Preserving recent file price_2021-08-10_2022-03-10_2022-03-15.feather
Preserving recent file price_2021-08-12_2022-03-14_2022-03-19.feather
Preserving recent file price_2021-08-11_2022-03-11_2022-03-16.feather
CPU times: user 1.95 s, sys: 1.5 s, total: 3.45 s
Wall time: 3.68 s


In [7]:
cols = ['name', 'close', 'pre5_pct_chg', 'pre20_pct_chg', 'circ_mv', 'total_mv', 'vol', 'turnover_rate_f', 
        'conseq_up_num', 'strth', 'first_time', 'last_time', 'amount', 'fd_amount', 'next_auc_amt', 
        'next_open_pct', 'next_auc_amt_ratio', 'next_pct_chg']

def get_stra(name, tdate):
    stra = Strategy(name, StockFilter(tdate).not_st().zb())
    stra.rules = {}
    stra.add_condition('conseq_up_num', '>=', val=1)
    stra.add_condition('conseq_up_num', '<=', val=3)
    stra.add_condition('amount',  '<', val=2000000) # 15亿，unit: k
    stra.add_condition('circ_mv', '<', val=1000000) # 80亿，unit: w
    stra.add_condition('turnover_rate_f', '<', val=45)
    stra.add_condition('turnover_rate_f', '>=', val=7)
    stra.add_condition('high', '>', var='low', ratio=1.02) # 日内振幅大于 2% (非一字板）
    stra.add_condition('vol_ratio', '>', val=1.5)
    stra.add_condition('vol', '>', var='ma_vol_5', ratio=1.7)
    
    # 次日竞价相关
    # stra.add_condition('next_open_pct', '>', val=0)
    # stra.add_condition('next_open_pct', '<', val=9)
    # stra.add_condition('next_auc_amt_ratio', '>', val=0.04) # 竞价金额占昨日成交金额百分比 大于 4%
    # stra.add_condition('next_auc_amt_ratio', '>', var='next_open_pct', ratio=0.01*3) # 竞价金额占昨日成交金额百分比 大于3倍 open_pct
    # stra.add_condition('next_auc_amt', '>', var='fd_amount', ratio=0.0002)
    return stra

In [13]:
%%time
cons = ak_all_plates(use_cache=True)
print(f'Total plate number: {len(cons)}')
con_sum, ind_sum, cons_detail = calc_plate_data(df, cons)
cons_today = cons_detail.xs(end_date, level='trade_date')
top_cons = calc_top_cons(cons_today=cons_today, cons=con_sum.xs(end_date, level='trade_date', drop_level=True))

Total plate number: 38743
CPU times: user 27 s, sys: 2.36 s, total: 29.3 s
Wall time: 30.6 s


In [14]:
stra1 = get_stra('new', end_date)
res, keys = stra1.get_result(df, end_date)
res.sort_values('conseq_up_num', inplace=True, ascending=False)
print(f'Found {len(res)}')
stock_summaries(df, res.index, end_date, get_auc=False, cols=up_cols, top_cons=top_cons)
display_up_df(res[UPSTOP_COLS])

Searching 2022-03-14...
Found 14
1. 皖通科技 (军工+举牌+智能交通, 2板 (W)，流值42.15亿，量比3.63，7.94亿，trf27.0%)：
2. 庄园牧场 (标普道琼斯A股+深股通+乳业, 2板 (O)，流值24.75亿，量比2.87，2.16亿，trf19.0%)：
3. 华盛昌 (人民币贬值受益+医疗器械概念+新冠检测, 2板 (T)，流值12.46亿，量比3.08，2.12亿，trf17.0%)：
4. 塞力医疗 (工业大麻+医疗器械概念+新冠检测, 2板 (T)，流值32.67亿，量比3.07，4.86亿，trf23.0%)：
5. 海南海药 (民营医院+生物疫苗+细胞免疫治疗, 1板 (W)，流值59.31亿，量比4.84，3.66亿，trf10.0%)：
6. 哈工智能 (军工+特斯拉+高端装备, 1板 (L)，流值50.13亿，量比1.84，4.03亿，trf11.0%)：
7. 安凯客车 (华为汽车+无人驾驶+摘帽, 1板 (W)，流值33.15亿，量比3.62，1.88亿，trf9.0%)：
8. 东港股份 (移动支付+蚂蚁金服概念+互联网彩票, 1板 (L)，流值51.0亿，量比4.09，4.06亿，trf12.0%)：
9. 宋都股份 (融资融券+土地流转+盐湖提锂, 1板 (W)，流值54.54亿，量比1.76，6.31亿，trf24.0%)：
10. 精伦电子 (语音技术+电子身份证+人脸识别, 1板 (L)，流值17.17亿，量比2.85，2.24亿，trf16.0%)：
11. 华纺股份 (人民币贬值受益+参股银行+口罩, 1板 (W)，流值23.37亿，量比5.7，3.36亿，trf18.0%)：
12. 贵航股份 (军工+通用航空, 1板 (L)，流值99.56亿，量比2.86，6.51亿，trf14.0%)：
13. 莱绅通灵 (新零售, 1板 (W)，流值30.64亿，量比1.74，1.92亿，trf22.0%)：
14. 金鸿顺 (年报预增, 1板 (W)，流值26.61亿，量比2.08，0.52亿，trf7.0%)：


,name,open,high,close,流值,总市值,vol,vol_type,vol_ratio,成交额,开盘涨幅,涨幅,up_type,first_time,last_time,open_times,strth,流动换手率,fl_ratio,fc_ratio
ts_code,,,,,,,,,,,,,,,,,,,,
002331.SZ,皖通科技,10.01,11.01,11.01,42.15亿,45.17亿,73.2万手,Ultra,3.63,7.94亿,0.00%,9.99%,W,09:34:24,14:54:36,50.00,52.23,27.49%,0.38,2.00
002910.SZ,庄园牧场,13.95,14.47,14.47,24.75亿,33.63亿,15.2万手,Huge,2.87,2.16亿,6.08%,10.04%,O,09:32:15,09:32:15,0.00,66.65,19.39%,1.53,17.48
002980.SZ,华盛昌,37.37,37.37,37.37,12.46亿,49.83亿,5.7万手,Huge,3.08,2.12亿,10.01%,10.01%,T,09:25:00,14:02:03,1.00,63.33,17.04%,3.35,19.66
603716.SH,塞力医疗,16.23,16.23,16.23,32.67亿,33.27亿,30.1万手,Huge,3.07,4.86亿,10.03%,10.03%,T,09:25:02,10:29:01,12.00,61.59,22.98%,0.73,4.88
000566.SZ,海南海药,4.63,5.09,5.09,59.31亿,66.04亿,74.1万手,Ultra,4.84,3.66亿,0.00%,9.94%,W,13:35:03,14:55:12,3.00,7.83,10.01%,0.08,1.33
000584.SZ,哈工智能,6.00,6.72,6.72,50.13亿,51.14亿,62.6万手,Huge,1.84,4.03亿,-1.80%,9.98%,L,13:20:30,13:20:30,0.00,51.80,11.13%,0.29,3.62
000868.SZ,安凯客车,4.12,4.52,4.52,33.15亿,33.15亿,42.2万手,Huge,3.62,1.88亿,0.24%,9.98%,W,09:40:00,14:54:36,7.00,48.78,9.38%,0.12,2.13
002117.SZ,东港股份,8.46,9.35,9.35,51.00亿,51.02亿,44.4万手,Huge,4.09,4.06亿,-0.47%,10.00%,L,10:06:27,11:28:03,1.00,49.97,11.76%,0.96,12.10
600077.SH,宋都股份,4.06,4.07,4.07,54.54亿,54.54亿,161.2万手,Huge,1.76,6.31亿,9.73%,10.00%,W,10:57:48,14:29:13,32.00,40.90,23.71%,0.09,0.81


In [15]:
def get_auc_by_tick(tdate, ts_codes=None):
    print(f'Fetching auc data as of {tdate}...')
    auc = pd.DataFrame()
    if ts_codes is None:
        ts_codes = get_stock_basic(tdate).index.to_list()
    for ts_code in tqdm(ts_codes):
        ts_code = get_ts_code_from_name(ts_code) if len(ts_code) != 6 and len(ts_code) != 9 else ts_code
        symbol = add_postfix(ts_code=ts_code, type='ak')
        df = tx_auc(symbol=symbol)
        if not df.empty:
            df.columns = ['time','price','price_diff', 'vol', 'amount', 'direction']
            tmpdf = pd.DataFrame()
            tmpdf.index.name = 'ts_code'
            tmpdf.loc[ts_code, 'open'] = df.iloc[0].price
            tmpdf.loc[ts_code, 'auc_amt'] = df.iloc[0].amount/1000
            tmpdf.loc[ts_code, 'auc_vol'] = df.iloc[0].vol/10
            tmpdf.loc[ts_code, 'trade_date'] = tdate
            auc = auc.append(tmpdf)
    insert_df(auc.reset_index(), 'auction')
    return auc

In [18]:
today_auc.columns.to_list()

['next_open', 'next_open_pct', 'next_auc_vol', 'next_auc_amt']

In [20]:
ts_codes = res.index.to_list()
if pdl.now().add(hours=18) < pdl.parse(end_date): # 没到18点, end_date 为前一日，取实时 tx_auc
    today_auc = get_auc_by_tick(today_date, ts_codes)   
    today_auc.loc[:,'open_pct'] = round((today_auc.next_open/today_auc.close-1)*100, 2)
else:
    today_auc = load_table(model=Auction, end_date=end_date, start_date=end_date)
    today_auc = today_auc.xs(end_date, level='trade_date', drop_level=True).drop(columns=['pre_close'])
today_auc.rename(columns={'open': 'next_open', 'auc_vol':'next_auc_vol', 'auc_amt': 'next_auc_amt', 'open_pct':'next_open_pct'}, inplace=True)
res = res.drop(columns=['next_open_pct']).join(today_auc)
res.loc[:,'next_auc_amt_ratio'] = round(res.next_auc_amt / res.amount, 2)
res

,open,high,low,close,pre_close,change,pct_chg,vol,amount,adj_factor,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,total_mv,circ_mv,total_share,float_share,free_share,last_factor,norm_adj,adj_vol,pre_close_4,pre_close_6,pre_close_11,pre_close_21,pre_pct_chg,pre3_pct_chg,pre5_pct_chg,pre10_pct_chg,pre20_pct_chg,next_close_1,next_close_2,next_close_3,next_close_10,next_close_20,next_pct_chg,next2_pct_chg,next3_pct_chg,next10_pct_chg,next20_pct_chg,max_pre10_price,min_pre10_price,max_pre20_price,min_pre20_price,max_pre30_price,min_pre30_price,max_pre120_price,min_pre120_price,avg_price,open_pct,ma_vol_5,ma_vol_20,vol_ratio,vol_ratio_long,vol_type,pre_vol_type,pre_trf,buy_sm_vol,buy_md_vol,buy_lg_vol,buy_elg_vol,sell_sm_vol,sell_md_vol,sell_lg_vol,sell_elg_vol,buy_sm_amount,buy_md_amount,buy_lg_amount,buy_elg_amount,sell_sm_amount,sell_md_amount,sell_lg_amount,sell_elg_amount,net_mf_vol,net_mf_amount,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit,name,list_date,upstop_num,conseq_up_num,pre_conseq_up_num,pre_open_times,pre3_upstops,pre5_upstops,pre10_upstops,pre20_upstops,post_up_num,up_type,pre_up_type,upstop_price,ma_close_5,ma_close_10,ma_close_30,ma_close_60,ma_close_120,ma_close_250,hl_pct,pre_ma_close_5,pre2_ma_close_5,pre3_ma_close_5,cur_date,list_days,bar_type,pre_bar_type,pre2_bar_type,cvo,pre_fl_ratio,pre_fc_ratio,pre_strth,pre_amount,pre_amp,next_cvo,next_open,next_open_pct,next_auc_vol,next_auc_amt,next_auc_amt_ratio
ts_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
002331.SZ,10.01,11.01,9.71,11.01,10.01,1.00,9.99,7.32e+05,793845.50,5.40,19.13,27.49,3.63,NaN,NaN,451680.78,421545.12,41024.59,38287.48,26633.41,5.40,1.0,7.32e+05,8.74,9.46,8.22,8.07,10.00,14.53,5.81,21.78,24.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.14,8.03,10.14,7.90,10.14,7.80,10.31,7.73,10.84,0.00,201904.74,118855.77,3.63,6.16,Ultra,Huge,15.80,165282.0,186481.0,161038.0,219471.0,182275.0,232894.0,157501.0,159603.0,17815.05,20145.09,17439.78,23984.63,19674.64,25175.32,17042.40,17492.19,-57209.0,-6161.40,12.99,2.00,0.38,1.58e+07,09:34:24,14:54:36,50.0,52.23,U,皖通科技,2010-01-06,1,2,1.0,2.0,2.0,2.0,3.0,3.0,NaN,W,L,11.01,9.58,9.34,8.60,8.59,8.70,NaN,11.81,9.16,9.05,9.13,2022-03-14,2960.0,yang,yang,cross,9.99e+00,1.71,16.11,49.50,406539.88,12.20,NaN,10.01,10.00,1144.5,11456.44,0.01
002910.SZ,13.95,14.47,13.79,14.47,13.15,1.32,10.04,1.52e+05,216422.12,1.01,8.91,19.39,2.87,321.67,92.09,336255.34,247526.64,23238.10,17106.19,7863.62,1.01,1.0,1.52e+05,11.69,11.92,11.64,11.18,10.04,12.49,10.32,12.97,17.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.15,11.30,13.15,11.13,13.15,10.38,13.15,9.80,14.20,6.08,53037.43,38678.32,2.87,3.94,Huge,Huge,12.23,36348.0,43185.0,32112.0,40799.0,26364.0,42579.0,44704.0,38797.0,5123.65,6106.12,4562.41,5850.04,3748.00,6038.67,6349.65,5505.89,-32339.0,-4733.35,5.17,17.48,1.53,3.78e+07,09:32:15,09:32:15,0.0,66.65,U,庄园牧场,2017-10-31,1,2,1.0,0.0,2.0,2.0,2.0,2.0,NaN,O,W,14.47,12.60,12.27,11.66,11.59,11.26,NaN,4.70,12.07,11.82,11.83,2022-03-14,1062.0,yang,yang,cross,3.95e+00,2.43,44.44,58.36,123060.54,11.21,NaN,13.95,16.74,1654.1,23074.69,0.11
002980.SZ,37.37,37.37,36.08,37.37,33.97,3.40,10.01,5.68e+04,212110.64,1.05,17.04,17.04,3.08,14.76,32.55,498266.90,124566.91,13333.34,3333.34,3333.34,1.05,1.0,5.68e+04,30.45,33.45,32.92,28.40,10.01,11.56,1.55,3.19,19.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.88,29.05,34.88,29.05,34.88,28.38,37.50,27.68,37.34,10.01,18440.03,20516.64,3.08,2.77,Huge,Normal,6.36,12272.0,13346.0,10262.0,20921.0,10627.0,13625.0,11898.0,20651.0,4578.47,4982.39,3833.00,7817.20,3968.19,5086.65,4440.79,7715.44,-45171.0,-16875.52,3.80,19.66,3.35,4.17e+07,09:25:00,14:02:03,1.0,63.33,U,华盛昌,2020-04-15,1,2,1.0,0.0,2.0,2.0,2.0,3.0,NaN,T,L,37.37,32.54,32.65,31.50,31.54,31.34,NaN,3.45,31.47,31.37,31.64,2022-03-14,464.0,cross,yang,cross,-3.13e-05,2.85,45.55,57.88,70771.99,11.82,NaN,37.37,21.02,1123.8,41996.41,0.20
603

In [21]:

display_up_df(res[cols])

,name,close,pre5_pct_chg,pre20_pct_chg,流值,总市值,vol,流动换手率,连板数,strth,first_time,last_time,成交额,封单金额,次日竞价金额,次开涨幅,next_auc_amt_ratio,次日涨幅
ts_code,,,,,,,,,,,,,,,,,,
002331.SZ,皖通科技,11.01,5.81,24.04,42.15亿,45.17亿,73.2万手,27.49%,2,52.23,09:34:24,14:54:36,7.94亿,0.16亿,0.11亿,10.00%,0.01,nan%
002910.SZ,庄园牧场,14.47,10.32,17.62,24.75亿,33.63亿,15.2万手,19.39%,2,66.65,09:32:15,09:32:15,2.16亿,0.38亿,0.23亿,16.74%,0.11,nan%
002980.SZ,华盛昌,37.37,1.55,19.61,12.46亿,49.83亿,5.7万手,17.04%,2,63.33,09:25:00,14:02:03,2.12亿,0.42亿,0.42亿,21.02%,0.20,nan%
603716.SH,塞力医疗,16.23,3.00,5.81,32.67亿,33.27亿,30.1万手,22.98%,2,61.59,09:25:02,10:29:01,4.86亿,0.24亿,0.43亿,21.03%,0.09,nan%
000566.SZ,海南海药,5.09,0.00,3.12,59.31亿,66.04亿,74.1万手,10.01%,1,7.83,13:35:03,14:55:12,3.66亿,0.05亿,0.01亿,3.35%,0.00,nan%
000584.SZ,哈工智能,6.72,6.26,15.72,50.13亿,51.14亿,62.6万手,11.13%,1,51.80,13:20:30,13:20:30,4.03亿,0.15亿,0.02亿,-1.80%,0.00,nan%
000868.SZ,安凯客车,4.52,1.48,6.20,33.15亿,33.15亿,42.2万手,9.38%,1,48.78,09:40:00,14:54:36,1.88亿,0.04亿,0.00亿,3.52%,0.00,nan%
002117.SZ,东港股份,9.35,-4.06,-6.18,51.00亿,51.02亿,44.4万手,11.76%,1,49.97,10:06:27,11:28:03,4.06亿,0.49亿,0.02亿,2.30%,0.00,nan%
600077.SH,宋都股份,4.07,1.37,29.82,54.54亿,54.54亿,161.2万手,23.71%,1,40.90,10:57:48,14:29:13,6.31亿,0.05亿,0.17亿,13.09%,0.03,nan%


In [32]:
yesterday_df = df.xs(yesterday_date, level='trade_date', drop_level=True)
yesterday_df

,open,high,low,close,pre_close,change,pct_chg,vol,amount,adj_factor,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,total_mv,circ_mv,total_share,float_share,free_share,last_factor,norm_adj,adj_vol,pre_close_4,pre_close_6,pre_close_11,pre_close_21,pre_pct_chg,pre3_pct_chg,pre5_pct_chg,pre10_pct_chg,pre20_pct_chg,next_close_1,next_close_2,next_close_3,next_close_10,next_close_20,next_pct_chg,next2_pct_chg,next3_pct_chg,next10_pct_chg,next20_pct_chg,max_pre10_price,min_pre10_price,max_pre20_price,min_pre20_price,max_pre30_price,min_pre30_price,max_pre120_price,min_pre120_price,avg_price,open_pct,next_open_pct,ma_vol_5,ma_vol_20,vol_ratio,vol_ratio_long,vol_type,pre_vol_type,pre_trf,buy_sm_vol,buy_md_vol,buy_lg_vol,buy_elg_vol,sell_sm_vol,sell_md_vol,sell_lg_vol,sell_elg_vol,buy_sm_amount,buy_md_amount,buy_lg_amount,buy_elg_amount,sell_sm_amount,sell_md_amount,sell_lg_amount,sell_elg_amount,net_mf_vol,net_mf_amount,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit,name,list_date,upstop_num,conseq_up_num,pre_conseq_up_num,pre_open_times,pre3_upstops,pre5_upstops,pre10_upstops,pre20_upstops,post_up_num,up_type,pre_up_type,upstop_price,ma_close_5,ma_close_10,ma_close_30,ma_close_60,ma_close_120,ma_close_250,hl_pct,pre_ma_close_5,pre2_ma_close_5,pre3_ma_close_5,cur_date,list_days,bar_type,pre_bar_type,pre2_bar_type,cvo,pre_fl_ratio,pre_fc_ratio,pre_strth,pre_amount,pre_amp,next_cvo
ts_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000001.SZ,14.40,14.98,14.15,14.90,14.57,0.33,2.2649,1493647.00,2187480.500,111.9210,0.7697,1.8303,1.04,7.9576,7.9576,2.891482e+07,2.891423e+07,1.940592e+06,1.940552e+06,816063.5000,111.9210,1.0,1493647.00,14.72,15.71,15.91,16.99,5.2746,-1.019022,-7.256525,-8.422376,-14.243673,14.49,NaN,NaN,NaN,NaN,-2.751678,NaN,NaN,NaN,NaN,16.08,13.22,17.34,13.22,17.56,13.22,20.99,13.22,14.65,-1.166781,-1.812081,1431409.140,1.030238e+06,1.043480,1.449808,Normal,Huge,2.7561,309354.0,416159.0,460073.0,308060.0,355807.0,525950.0,400485.0,211406.0,45215.36,60854.51,67323.00,45355.19,52189.84,77134.45,58601.23,30822.55,223652.0,33215.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,平安银行,1991-04-03,0,0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,16.03,14.468,15.073,16.031000,16.608833,17.595333,NaN,5.570470,14.554,14.782,15.150,2022-03-11,7557.0,yang,yang,yin,3.431681,NaN,NaN,NaN,3269867.800,NaN,-0.939619
000002.SZ,16.35,16.75,16.02,16.65,16.77,-0.12,-0.7156,1021248.40,1676581.400,162.7380,1.0509,1.5693,0.62,4.6624,5.0484,1.935626e+07,1.617973e+07,1.162538e+06,9.717553e+05,650774.2500,162.7380,1.0,1021248.40,18.54,19.49,19.84,20.85,-1.2367,-9.546926,-13.955875,-15.473790,-19.568345,16.09,NaN,NaN,NaN,NaN,-3.363363,NaN,NaN,NaN,NaN,20.03,16.05,21.68,16.05,22.07,16.05,23.09,16.05,16.42,-2.504472,-1.621622,1654708.400,1.170340e+06,0.617177,0.872608,Normal,Normal,1.6274,276591.0,334190.0,298832.0,111635.0,181942.0,348888.0,339940.0,150478.0,45382.44,54846.10,49112.28,18317.32,29919.83,57323.72,55775.77,24638.83,55006.0,9154.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,万科A,1991-01-29,0,0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,18.45,17.322,18.271,19.745000,20.147833,19.962417,NaN,4.384384,17.734,18.278,18.746,2022-03-11,7601.0,yang,yin,yin,1.788872,NaN,NaN,NaN,1796610.800,NaN,-1.741778
000004.SZ,18.70,19.58,18.55,19.51,18.93,0.58,3.0639,54527.32,104705.990,4.0640,4.6873,6.7195,1.00,47.0980,74.3818,3.043619e+05,2.269613e+05,1.560030e+04,1.163308e+04,8114.8430,4.0640,1.0,54527.32,20.99,21.24,20.64,22.56,-0.7862,-9.814197,-10.875706,-8.284884,-16.090426,19.32,NaN,NaN,NaN,NaN,-0.973860,NaN,NaN,NaN,NaN,22.28,18.16,25.86,18.16,25.86,18.16,25.86,16.20,19.20,-1.215003,0.461302,54480.500,9.634758e+04,1.000859,0.565944,Small,Small,6.8372,26454.0,20174.0,6341.0,1558.0,24382.0,19982.0,7097.0,3066.0,5076.74,3873.12,1221.16,299.58,4677.39,3834.08,1367.01,592.12,7655.0,1478.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,国华网安,1991-01-14,0,0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,20

In [49]:
auc_df = yesterday_df.join(today_auc)
auc_df.loc[:,'open_pct'] = round((auc_df.next_open/auc_df.close-1)*100, 2)
auc_df.loc[:,'next_auc_amt_ratio'] = round(auc_df.next_auc_amt / auc_df.amount, 2)
auc_df

,open,high,low,close,pre_close,change,pct_chg,vol,amount,adj_factor,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,total_mv,circ_mv,total_share,float_share,free_share,last_factor,norm_adj,adj_vol,pre_close_4,pre_close_6,pre_close_11,pre_close_21,pre_pct_chg,pre3_pct_chg,pre5_pct_chg,pre10_pct_chg,pre20_pct_chg,next_close_1,next_close_2,next_close_3,next_close_10,next_close_20,next_pct_chg,next2_pct_chg,next3_pct_chg,next10_pct_chg,next20_pct_chg,max_pre10_price,min_pre10_price,max_pre20_price,min_pre20_price,max_pre30_price,min_pre30_price,max_pre120_price,min_pre120_price,avg_price,open_pct,next_open_pct,ma_vol_5,ma_vol_20,vol_ratio,vol_ratio_long,vol_type,pre_vol_type,pre_trf,buy_sm_vol,buy_md_vol,buy_lg_vol,buy_elg_vol,sell_sm_vol,sell_md_vol,sell_lg_vol,sell_elg_vol,buy_sm_amount,buy_md_amount,buy_lg_amount,buy_elg_amount,sell_sm_amount,sell_md_amount,sell_lg_amount,sell_elg_amount,net_mf_vol,net_mf_amount,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit,name,list_date,upstop_num,conseq_up_num,pre_conseq_up_num,pre_open_times,pre3_upstops,pre5_upstops,pre10_upstops,pre20_upstops,post_up_num,up_type,pre_up_type,upstop_price,ma_close_5,ma_close_10,ma_close_30,ma_close_60,ma_close_120,ma_close_250,hl_pct,pre_ma_close_5,pre2_ma_close_5,pre3_ma_close_5,cur_date,list_days,bar_type,pre_bar_type,pre2_bar_type,cvo,pre_fl_ratio,pre_fc_ratio,pre_strth,pre_amount,pre_amp,next_cvo,next_open,next_auc_vol,next_auc_amt,next_auc_amt_ratio
ts_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000001.SZ,14.40,14.98,14.15,14.90,14.57,0.33,2.2649,1493647.00,2187480.500,111.9210,0.7697,1.8303,1.04,7.9576,7.9576,2.891482e+07,2.891423e+07,1.940592e+06,1.940552e+06,816063.5000,111.9210,1.0,1493647.00,14.72,15.71,15.91,16.99,5.2746,-1.019022,-7.256525,-8.422376,-14.243673,14.49,NaN,NaN,NaN,NaN,-2.751678,NaN,NaN,NaN,NaN,16.08,13.22,17.34,13.22,17.56,13.22,20.99,13.22,14.65,-1.81,-1.812081,1431409.140,1.030238e+06,1.043480,1.449808,Normal,Huge,2.7561,309354.0,416159.0,460073.0,308060.0,355807.0,525950.0,400485.0,211406.0,45215.36,60854.51,67323.00,45355.19,52189.84,77134.45,58601.23,30822.55,223652.0,33215.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,平安银行,1991-04-03,0,0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,16.03,14.468,15.073,16.031000,16.608833,17.595333,NaN,5.570470,14.554,14.782,15.150,2022-03-11,7557.0,yang,yang,yin,3.431681,NaN,NaN,NaN,3269867.800,NaN,-0.939619,14.63,975.6,14273.028,0.01
000002.SZ,16.35,16.75,16.02,16.65,16.77,-0.12,-0.7156,1021248.40,1676581.400,162.7380,1.0509,1.5693,0.62,4.6624,5.0484,1.935626e+07,1.617973e+07,1.162538e+06,9.717553e+05,650774.2500,162.7380,1.0,1021248.40,18.54,19.49,19.84,20.85,-1.2367,-9.546926,-13.955875,-15.473790,-19.568345,16.09,NaN,NaN,NaN,NaN,-3.363363,NaN,NaN,NaN,NaN,20.03,16.05,21.68,16.05,22.07,16.05,23.09,16.05,16.42,-1.62,-1.621622,1654708.400,1.170340e+06,0.617177,0.872608,Normal,Normal,1.6274,276591.0,334190.0,298832.0,111635.0,181942.0,348888.0,339940.0,150478.0,45382.44,54846.10,49112.28,18317.32,29919.83,57323.72,55775.77,24638.83,55006.0,9154.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,万科A,1991-01-29,0,0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,18.45,17.322,18.271,19.745000,20.147833,19.962417,NaN,4.384384,17.734,18.278,18.746,2022-03-11,7601.0,yang,yin,yin,1.788872,NaN,NaN,NaN,1796610.800,NaN,-1.741778,16.38,638.1,10452.078,0.01
000004.SZ,18.70,19.58,18.55,19.51,18.93,0.58,3.0639,54527.32,104705.990,4.0640,4.6873,6.7195,1.00,47.0980,74.3818,3.043619e+05,2.269613e+05,1.560030e+04,1.163308e+04,8114.8430,4.0640,1.0,54527.32,20.99,21.24,20.64,22.56,-0.7862,-9.814197,-10.875706,-8.284884,-16.090426,19.32,NaN,NaN,NaN,NaN,-0.973860,NaN,NaN,NaN,NaN,22.28,18.16,25.86,18.16,25.86,18.16,25.86,16.20,19.20,0.46,0.461302,54480.500,9.634758e+04,1.000859,0.565944,Small,Small,6.8372,26454.0,20174.0,6341.0,1558.0,24382.0,19982.0,7097.0,3066.0,5076.74,3873.12,1221.16,299.58,4677.39,3834.08,1367.01,592.12,7655.0,14

In [ ]:
auc_df.max_pre120_price # {max, min} 10, 20, 30, 120
auc_df.ma_vol_20 #5, 20
auc_df.pre20_upstops # 5, 10, 20
auc_df.pre20_pct_chg # 10, 20
auc_df.ma_close_5 # [5, 10, 30, 60, 120, 250]

In [67]:
res[['fd_amount','next_auc_amt']]

,fd_amount,next_auc_amt
ts_code,,
002246.SZ,3.57e+07,83197.10
002910.SZ,5.47e+07,23074.69
002932.SZ,1.20e+08,129532.80
600327.SH,3.71e+07,18890.72
600676.SH,1.74e+07,19444.80
600736.SH,6.46e+07,57219.75
603636.SH,2.34e+07,94358.08


In [85]:
res, _ = stra1.get_result(df=auc_df)
display_up_df(res[cols])

,name,close,pre5_pct_chg,pre20_pct_chg,流值,总市值,vol,流动换手率,连板数,strth,first_time,last_time,成交额,封单金额,次日竞价金额,次开涨幅,next_auc_amt_ratio,次日涨幅
ts_code,,,,,,,,,,,,,,,,,,
000505.SZ,京粮控股,7.95,-8.13,0.28,49.35亿,57.79亿,39.0万手,13.73%,1,49.09,10:20:15,11:18:48,3.03亿,0.37亿,0.17亿,1.13%,0.05,-2.14%
002910.SZ,庄园牧场,13.15,-0.42,4.92,22.49亿,30.56亿,9.6万手,12.23%,1,58.36,09:43:51,09:43:51,1.23亿,0.55亿,0.23亿,6.08%,0.19,10.04%
